### Import the functions written in the query_helper.py file

In [1]:
!pip install regex

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install psycopg2

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import psycopg2 as pg2
import pandas as pd
import re

### Write Functions to Query from the Tables Created

In [4]:
from os import chdir
chdir('/home/jovyan/')

In [5]:
import library.db_helper as db
import library.functions as fy

In [22]:
con, cur = db.connect_to_db()

### Insert content into the created tables (which are currently empty)

In [31]:
# define function to pass in category_name and category_id

def insert_category (id_no, name):
    '''
    grab category names and ids and push into the postsql db
    '''
    db.connect_to_db()
    con, cur = db.connect_to_db()
    
    query = '''
            BEGIN;
            INSERT INTO category VALUES ({}, '{}');
            COMMIT;
            '''.format(id_no, name)
    
    cur.execute(query)

In [7]:
CATEGORY_df = pd.read_pickle('./data/CATEGORY_df.p')

In [8]:
CATEGORY_df.reset_index(inplace=True, drop=True)

In [9]:
CATEGORY_df.head()

,category,categoryid
0,Category:Machine_learning,43
1,Category:Applied machine learning,1
2,Category:Artificial neural networks,3
3,Category:Bayesian networks,4
4,Category:Classification algorithms,12


In [10]:
CATEGORY_df['category'] = [category[9:] for category in CATEGORY_df['category']]
# drop the word "Category:" from my category names before pushing it into my PostgreSQL table

In [11]:
CATEGORY_df.head()

,category,categoryid
0,Machine_learning,43
1,Applied machine learning,1
2,Artificial neural networks,3
3,Bayesian networks,4
4,Classification algorithms,12


In [17]:
CATEGORY_df['category'] = CATEGORY_df['category'].apply(lambda x: fy.text_cleaner(x))

In [18]:
CATEGORY_df.head()

,category,categoryid
0,machine learning,43
1,applied machine learning,1
2,artificial neural networks,3
3,bayesian networks,4
4,classification algorithms,12


In [23]:
# clean out the category db that was previously created 

delete_cat_query = '''
BEGIN;
DELETE FROM category;
COMMIT;
'''

cur.execute(delete_cat_query)

In [24]:
# query = '''
# SELECT *
# FROM category
# LIMIT 1
# '''
# 
# cur.execute(query)

In [32]:
for i in CATEGORY_df.index:
    insert_category(CATEGORY_df.iloc[i,1], CATEGORY_df.iloc[i,0])
    

In [36]:
query = '''
SELECT * 
FROM category
'''

db.query_to_dataframe(query)

,cid,name
0,43,machine learning
1,1,applied machine learning
2,3,artificial neural networks
3,4,bayesian networks
4,12,classification algorithms
5,13,cluster analysis
6,15,computational learning theory
7,2,artificial intelligence conferences
8,16,data mining and machine learning software
9,17,datasets in machine learning


In [37]:
def insert_page (id_no, title, text):
    '''
    grab page ids, titles, and text, and then push into the postsql db
    '''
    db.connect_to_db()
    con, cur = db.connect_to_db()
    
    query = '''
            BEGIN;
            INSERT INTO page VALUES ({}, '{}', '{}');
            COMMIT;
            '''.format(id_no, title, text)
    
    cur.execute(query)

In [38]:
PAGE_df = pd.read_pickle('./data/PAGE_df.p')

In [39]:
PAGE_df.head()

,pageid,title,text
0,54972729,User:CustIntelMngt/sandbox/Customer Intelligen...,this is not a wikipedia article it is an indiv...
1,43385931,Data exploration,this article has multiple issues please help i...
2,49082762,List of datasets for machine learning research,machine learning anddata miningproblemsclassif...
3,233488,Machine learning,for the journal see machine learning journal m...
4,53587467,Outline of machine learning,the following outline is provided as an overvi...


In [66]:
import re

In [40]:
PAGE_df['title'] = PAGE_df['title'].apply(lambda x: fy.text_cleaner(x))

In [41]:
PAGE_df.shape

(2787, 3)

In [42]:
PAGE_df.drop_duplicates('title', inplace=True)

In [44]:
PAGE_df.shape

(2439, 3)

In [47]:
PAGE_df.reset_index(inplace=True)

In [55]:
delete_page_query = '''
BEGIN;
DELETE FROM page;
COMMIT;
'''

cur.execute(delete_page_query)

In [56]:
for i in PAGE_df.index:
    insert_page(PAGE_df.iloc[i,0], PAGE_df.iloc[i,1], PAGE_df.iloc[i,2])

In [57]:
query = '''
SELECT * 
FROM page
'''

db.query_to_dataframe(query).shape

(2439, 3)

In [23]:
# REVISIT LATER!
# df.apply(lambda x: insert_category(x.name, x.cid), axis=1)
# look at documentation for lambda and axes to quadruple check, for my own edification also

# CATEGORY_df


NameError: name 'category' is not defined

In [58]:
total_df = pd.read_pickle('./data/total_df.p')

In [62]:
from sklearn.preprocessing import LabelEncoder

In [63]:
total_df.head()

,ns,pageid,title,category,text
0,2,54972729,User:CustIntelMngt/sandbox/Customer Intelligen...,Category:Machine_learning,this is not a wikipedia article it is an indiv...
1,0,43385931,Data exploration,Category:Machine_learning,this article has multiple issues please help i...
2,0,49082762,List of datasets for machine learning research,Category:Machine_learning,machine learning anddata miningproblemsclassif...
3,0,233488,Machine learning,Category:Machine_learning,for the journal see machine learning journal m...
4,0,53587467,Outline of machine learning,Category:Machine_learning,the following outline is provided as an overvi...


In [64]:
le = LabelEncoder()

In [65]:
total_df['category_id'] = le.fit_transform(total_df['category'])

In [66]:
total_df.drop('ns', axis=1, inplace=True)

In [67]:
total_df.head()

,pageid,title,category,text,category_id
0,54972729,User:CustIntelMngt/sandbox/Customer Intelligen...,Category:Machine_learning,this is not a wikipedia article it is an indiv...,43
1,43385931,Data exploration,Category:Machine_learning,this article has multiple issues please help i...,43
2,49082762,List of datasets for machine learning research,Category:Machine_learning,machine learning anddata miningproblemsclassif...,43
3,233488,Machine learning,Category:Machine_learning,for the journal see machine learning journal m...,43
4,53587467,Outline of machine learning,Category:Machine_learning,the following outline is provided as an overvi...,43


In [68]:
CATEGORY_PAGE_df = total_df[['pageid', 'category_id']]

In [69]:
CATEGORY_PAGE_df.shape

(2787, 2)

In [81]:
CATEGORY_PAGE_df.columns

Index(['pageid', 'category_id'], dtype='object')

In [93]:
def insert_category_page (page_id, category_id):
    con, cur = connect_to_db()
    
    query = '''
    BEGIN;
    INSERT INTO category_page VALUES ('{}', '{}');
    COMMIT;
    '''.format(page_id, category_id)
    
    cur.execute(query)

In [90]:
CATEGORY_PAGE_df.iloc[0, 1]

43

In [94]:
for i in CATEGORY_PAGE_df.index:
    insert_category_page(CATEGORY_PAGE_df.iloc[i,0], CATEGORY_PAGE_df.iloc[i,1])

In [72]:
query = '''
SELECT *
FROM category_page
'''

db.query_to_dataframe(query)

,category_cid,page_pid
0,43,54972729
1,43,43385931
2,43,49082762
3,43,233488
4,43,53587467
5,43,3771060
6,43,43808044
7,43,28801798
8,43,45049676
9,43,52642349
